In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
from Metrics_v2 import StrokeMetrics, ALSMetrics, ALSorStrokeMetrics
import pandas as pd
import numpy as np
import argparse
import os
from typing import Dict, List

In [6]:
# Some helper functions

# Audio tasks use an audio.Table while others use a video.Table
audio_tasks = ["BBP_NORMAL"]

# Used to extract the patient type from the landmark file name
ids = {
    "stroke": ["OP", "S"],
    "healthy": ["N"],
    "als": ["A"]
}

# Defines which metric calculator to use based on the 'metric_type' argument
metric_calc_map = {
    "stroke": StrokeMetrics,
    "als": ALSMetrics,
    "all": ALSorStrokeMetrics
}


def compute_metrics(inputs, out_path, metrics_type = 'all'):
    """
    Computes the metrics for every file in the data csv
    :param inputs: A dataframe with columns: csv_path, rep_{}_start,
                                                       rep_{}_end
    :param out_path: A path to a csv to store the metric data in
    """
    # Key is task name and value is all repetitions of the task
    metric_frames: Dict[str, List[pd.DataFrame]] = {}
    for index, row in inputs.iterrows():
        # Read all data from the input dataframe
        try:
            data_path = row['csv_path']
            rest_path = row['rest_path']
            data_frame = pd.read_csv(data_path)
            rest_frame = pd.read_csv(rest_path)
            task_type = row['task']
            subject_id = row['subject']
            subject_type = row['type']
        except KeyError:
            raise KeyError("Input csv must contain 'csv_path', 'rest_path', 'rest_table_path' \
            'task', 'subject', and 'type' columns. \nFurthermore, 'csv_path' \
            and 'rest_path' must be files on the disk.")
        print("Starting:", data_path, "with", subject_type, "patients")
        rep_ranges = []
        rep_i = 1
        # Grab data about the start and end of repetitions
        while f"rep_{rep_i}_start" in row and f"rep_{rep_i}_end" in row:
            try:
                start_val = row[f"rep_{rep_i}_start"]
                end_val = row[f"rep_{rep_i}_end"]
                rep_ranges.append(range(int(start_val), int(end_val)))
            except ValueError as e:
                pass
            rep_i += 1
        rep_i -= 1

        # Calculate metrics
        metric_calc = metric_calc_map[metrics_type](data_frame, rest_frame)
        # metric_calc = StrokeMetrics(data_frame, rest_frame)
        metric_frame = None
        for rep_range in rep_ranges:
            metrics = metric_calc.compute_metrics(active_frames=rep_range)
            if metric_frame is None:
                metric_frame = metrics
            else:
                metric_frame = pd.concat([metric_frame, metrics], ignore_index=True)
        rep_nums = list(range(1, rep_i+1))

        # Insert metadata to final csv
        metric_frame.insert(0, 'rep', pd.Series(rep_nums))
        metric_frame.insert(0, 'type', pd.Series(np.full(rep_i, int(not(subject_type == "healthy")))))
        metric_frame.insert(0, 'task', pd.Series(np.full(rep_i, task_type)))
        metric_frame.insert(0, 'subject_id', pd.Series(np.full(rep_i, subject_id)))
        try:
            metric_frames[task_type].append(metric_frame)
        except KeyError:
            metric_frames[task_type] = [metric_frame]

    # save data to disk
    for task, metrics in metric_frames.items():
        csv_name = "metric_test_{}.csv".format(task)
        stand_csv_name = "metric_output_{}_standardized.csv".format(task)
        all_metrics_frame = pd.concat(metrics, ignore_index=True)

        # Create a new frame standardized by z-score of columns
        all_metrics_stand = all_metrics_frame.copy()
        for col in all_metrics_stand.columns:
            if col in ['subject_id', 'type', 'task', 'rep']:
                continue
            feature = all_metrics_stand[col]
            mean = np.mean(feature)
            sd = np.std(feature)
            if sd != 0:
                z_scores = (feature - mean) / sd
            else:
                z_scores = ["NA" for i in range(len(feature))]
            all_metrics_stand[col] = z_scores
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        all_metrics_frame.to_csv(os.path.join(out_path, csv_name))
        all_metrics_stand.to_csv(os.path.join(out_path, stand_csv_name))
    print(f"Saved metrics to {os.path.abspath(out_path)}")
    return metric_frames


def convert_time_to_frames(time_file, landmarks_file):
    """
    Converts audio timestamps into video frames
    :param time_file:
    :param landmarks_file:
    :return:
    """
    data = pd.read_csv(time_file, index_col=None)
    landmarks_DF = pd.read_csv(landmarks_file, index_col=0)
    tmin = data[data.columns[0]].values
    tmax = data[data.columns[-1]].values
    vid_time = landmarks_DF['Time_Stamp (s)'].values[1:]
    positions = np.zeros((len(tmin), 2), dtype=int)
    for k, (mini, maxi) in enumerate(zip(tmin, tmax)):
        positions[k, 0] = int(np.argmin(abs(vid_time - mini)))
        positions[k, 1] = int(np.argmin(abs(vid_time - maxi)))
    new_df = pd.DataFrame()
    new_df['Tstart'] = positions[:, 0]
    new_df['text'] = data['text']
    new_df['Tend'] = positions[:, 1]
    return new_df

# Main Program 
You need to provide a .csv file with three columns : 'landmarks'  'landmarks_table'   'rest' 


- the 'landmarks' column contains the path to the .csv file that contains the landmark positions for the task under analysis (either 2d or 3d)
- the 'landmarks_table' column cotains the path to the .table file that contains the information about the repetitions either as frames 
  (_video.table) or as time stamps (_audio.table). When using time stamps, there has to be a 'FrameInfoColor.csv' file that contains the 
  relation between time stamps and frame number
- The 'rest' column contains the path to the .csv file that contains the landmark positions for the REST task (either 2d or 3d)


NOTE: There has to be a 'REST' file associated with the subject 
### The program won't continue if the REST file is not provided

In [13]:

def save_results(input_csv, output_folder, metric_type):
    analysis_csv = pd.DataFrame()
    files = pd.read_csv(input_csv)
    for i, row in files.iterrows():
        # Construct a dataframe with the information necessary for processing
        landmark_path = os.path.abspath(row["landmarks"])
        landmark_file_name = os.path.basename(landmark_path)
        table_path = os.path.abspath(row["landmarks_table"])
        table_file_name = os.path.basename(table_path)
        rest_path = os.path.abspath(row["rest"])
        rest_file_name = os.path.basename(table_path)
        print(landmark_path)
        print()
        print(rest_path)
        print()
        print(table_path)

        if not os.path.isfile(landmark_path):
            raise RuntimeError("Landmark path is not a file on disk")
        if not os.path.isfile(rest_path):
            raise RuntimeError("Rest file is not a file on disk")
        if not os.path.isfile(table_path):
            raise RuntimeError("Table file is not a file on disk")

        landmark_file_data = landmark_file_name.split("_")
        subject = landmark_file_data[0]
        subject_type = ""
        # Check if a user fits into a known subject type
        for s_type, prefixes in ids.items():
            for prefix in prefixes:
                if prefix in subject:
                    subject_type = s_type
        task = "_".join(landmark_file_data[2:4])
        parsed_data = pd.DataFrame(columns=["csv_path", "type", "rest_path", "subject", "task"])
        parsed_data.loc[0] = 0

        # If it is an audio task, convert the table file to frame numbers
        if 'audio' in table_file_name:
            video_info = convert_time_to_frames(table_path, landmark_path)
        else:
            video_info = pd.read_csv(table_path)


        parsed_data["csv_path"] = landmark_path
        parsed_data["rest_path"] = rest_path
        parsed_data["type"] = subject_type
        parsed_data["subject"] = subject
        parsed_data["task"] = task
        for index, row in video_info.iterrows():
            parsed_data[f"rep_{index+1}_start"] = row[video_info.columns[0]]
            parsed_data[f"rep_{index+1}_end"] = row[video_info.columns[-1]] 
        analysis_csv = analysis_csv.append(parsed_data, ignore_index=True, sort=False)


    print(analysis_csv)

    compute_metrics(analysis_csv, output_folder,metric_type);

In [14]:
input_csv = r'/media/deniz/easystore/facial_tracking_data_2021/mobileNet_3D_lookups.csv'
df= pd.read_csv(input_csv)
df = df.drop('Unnamed: 0', axis=1)
df.head()

,subject,landmarks,landmarks_table,rest,task
0,AF01,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,OPEN
1,AF03,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,SPREAD
2,AF10,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,OPEN
3,AF10,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,SPREAD
4,AF11,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,OPEN


In [15]:
metric_type = 'stroke' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'


#input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_New_Table.csv" #csv file with landmarks and .TABLE info
output_folder = r"/home/deniz/Code/ClinicalScore/results" #where to store the results
save_results(input_csv, output_folder, metric_type)

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/mobileNet/AF01_02_20170509_NSM_OPEN_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/mobileNet/AF01_02_20170509_RST_REST_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/TABLE/AF01_02_20170509_NSM_OPEN_video.Table
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF03_SBK_20170526/02/LANDMARKS/mobileNet/AF03_02_20170526_NSM_SPREAD_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF03_SBK_20170526/02/LANDMARKS/mobileNet/AF03_02_20170526_RST_REST_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF03_SBK_20170526/02/TABLE/AF03_02_20170526_NSM_SPREAD_video.Table
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF10_SBK_20170613/02/LANDMARKS/mobileNet/AF10_02_20170613_NSM_OPEN_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/LANDMARKS/mobileNet/OP01_02_20190510_NSM_KISS_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/LANDMARKS/mobileNet/OP01_02_20190510_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/TABLE/OP01_02_20190510_NSM_KISS_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/LANDMARKS/mobileNet/OP01_02_20190510_NSM_OPEN_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/LANDMARKS/mobileNet/OP01_02_20190510_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/TABLE/OP01_02_20190510_NSM_OPEN_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/LANDMARKS/mobileNet/OP01_02_20190510_NSM_SPREAD_color_landma

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP03_SBK_20190524/02/LANDMARKS/mobileNet/OP03_02_20190524_NSM_SPREAD_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP03_SBK_20190524/02/LANDMARKS/mobileNet/OP03_02_20190524_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP03_SBK_20190524/02/TABLE/OP03_02_20190524_NSM_SPREAD_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/mobileNet/OP05_02_20190719_NSM_BIGSMILE_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/mobileNet/OP05_02_20190719_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/TABLE/OP05_02_20190719_NSM_BIGSMILE_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/mobileNet/OP05_02_20190719_NSM_OPEN_co

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20190906/02/LANDMARKS/mobileNet/OP08_02_20190906_NSM_OPEN_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20190906/02/LANDMARKS/mobileNet/OP08_02_20190906_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20190906/02/TABLE/OP08_02_20190906_NSM_OPEN_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20190906/02/LANDMARKS/mobileNet/OP08_02_20190906_NSM_SPREAD_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20190906/02/LANDMARKS/mobileNet/OP08_02_20190906_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20190906/02/TABLE/OP08_02_20190906_NSM_SPREAD_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20191023/02/LANDMARKS/mobileNet/OP08_02_20191023_NSM_BIGSMILE_color_

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP11_SBK_20191118/02/LANDMARKS/mobileNet/OP11_02_20191118_NSM_BIGSMILE_1_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP11_SBK_20191118/02/LANDMARKS/mobileNet/OP11_02_20191118_RST_REST2_1_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP11_SBK_20191118/02/TABLE/OP11_02_20191118_NSM_BIGSMILE_1_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP11_SBK_20191118/02/LANDMARKS/mobileNet/OP11_02_20191118_NSM_OPEN_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP11_SBK_20191118/02/LANDMARKS/mobileNet/OP11_02_20191118_RST_REST2_1_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP11_SBK_20191118/02/TABLE/OP11_02_20191118_NSM_OPEN_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP11_SBK_20191118/02/LANDMARKS/mobileNet/OP11_02_20191118_NSM_SPR

...

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF08_SBK_20170615/02/LANDMARKS/mobileNet/NF08_02_20170615_NSM_SPREAD_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF08_SBK_20170615/02/LANDMARKS/mobileNet/NF08_02_20170615_RST_REST_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF08_SBK_20170615/02/TABLE/NF08_02_20170615_NSM_SPREAD_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF09_SBK_20171012/02/LANDMARKS/mobileNet/NF09_02_20171012_NSM_OPEN_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF09_SBK_20171012/02/LANDMARKS/mobileNet/NF09_02_20171012_RST_REST_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF09_SBK_20171012/02/TABLE/NF09_02_20171012_NSM_OPEN_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF09_SBK_20171012/02/LANDMARKS/mobileNet/NF09_02_20

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF19_TRI_20190627/02/LANDMARKS/mobileNet/NF19_02_20190627_NSM_BIGSMILE_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF19_TRI_20190627/02/LANDMARKS/mobileNet/NF19_02_20190627_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF19_TRI_20190627/02/TABLE/NF19_02_20190627_NSM_BIGSMILE_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF19_TRI_20190627/02/LANDMARKS/mobileNet/NF19_02_20190627_NSM_OPEN_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF19_TRI_20190627/02/LANDMARKS/mobileNet/NF19_02_20190627_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF19_TRI_20190627/02/TABLE/NF19_02_20190627_NSM_OPEN_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF19_TRI_20190627/02/LANDMARKS/mobileNet/NF19

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF25_SBK_20190815/02/LANDMARKS/mobileNet/NF25_02_20190815_NSM_PUCKER_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF25_SBK_20190815/02/LANDMARKS/mobileNet/NF25_02_20190815_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF25_SBK_20190815/02/TABLE/NF25_02_20190815_NSM_PUCKER_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF25_SBK_20190815/02/LANDMARKS/mobileNet/NF25_02_20190815_NSM_SPREAD_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF25_SBK_20190815/02/LANDMARKS/mobileNet/NF25_02_20190815_RST_REST2_color_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF25_SBK_20190815/02/TABLE/NF25_02_20190815_NSM_SPREAD_video.Table
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF26_TRI_20190829/02/LANDMARKS/mobileNet/NF26

Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF14_SBK_20190129/02/LANDMARKS/mobileNet/AF14_02_20190129_NSM_OPEN_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF14_SBK_20190129/02/LANDMARKS/mobileNet/AF14_02_20190129_NSM_SPREAD_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF16_SBK_20190403/02/LANDMARKS/mobileNet/AF16_02_20190403_NSM_BIGSMILE_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF16_SBK_20190403/02/LANDMARKS/mobileNet/AF16_02_20190403_NSM_KISS_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF16_SBK_20190403/02/LANDMARKS/mobileNet/AF16_02_20190403_NSM_OPEN_color_landmarks3D.csv with als patients


IndexError: index 280 is out of bounds for axis 0 with size 280

In [5]:
input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_vs_1Control_finetuned.csv"

df= pd.read_csv(input_csv)
df.head()

,landmarks,landmarks_table,rest
0,E:\FaceData_\video_data\ALS\old\A001\landmarks...,E:\FaceData_\video_data\ALS\old\A001\A001_02_B...,E:\FaceData_\video_data\ALS\old\A001\landmarks...
1,E:\FaceData_\video_data\ALS\old\A001\landmarks...,E:\FaceData_\video_data\ALS\old\A001\A001_02_N...,E:\FaceData_\video_data\ALS\old\A001\landmarks...
2,E:\FaceData_\video_data\ALS\old\A001\landmarks...,E:\FaceData_\video_data\ALS\old\A001\A001_02_N...,E:\FaceData_\video_data\ALS\old\A001\landmarks...
3,E:\FaceData_\video_data\ALS\old\A004\landmarks...,E:\FaceData_\video_data\ALS\old\A004\A004_02_B...,E:\FaceData_\video_data\ALS\old\A004\landmarks...
4,E:\FaceData_\video_data\ALS\old\A004\landmarks...,E:\FaceData_\video_data\ALS\old\A004\A004_02_N...,E:\FaceData_\video_data\ALS\old\A004\landmarks...


In [6]:
def change_path(df_cell):

    df_cell = df_cell.replace("E:\FaceData_", "\\Users\denizjafari\documents\CODE\ClinicalScore\FaceData_")
    df_cell = df_cell.replace("\\", "/")
    return df_cell

In [7]:
# change the path to the correct path on my MAC 
try:
    df['landmarks'] = df['landmarks'].apply(lambda x: change_path(x))
    df['landmarks_table'] = df['landmarks_table'].apply(lambda x: change_path(x))
    df['rest'] = df['rest'].apply(lambda x: change_path(x))
except:
    pass

# Write the results into csv
df.to_csv('ALS_New_Table.csv')
try:
    df = df.drop(['Unnamed: 0'], axis=1)
except:
    pass

df.head()

,landmarks,landmarks_table,rest
0,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
1,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
2,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
3,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
4,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...


In [8]:
df_test = df[0:1]
df_test.to_csv('Test_Table.csv')
df_test

,landmarks,landmarks_table,rest
0,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...


In [9]:
metric_type = 'als' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'


#finetuned

input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/Test_Table.csv"
#input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_New_Table.csv" #csv file with landmarks and .TABLE info
output_folder = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/results/TEST" #where to store the results
save_results(input_csv, output_folder, metric_type)



/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_BBP_NORMAL_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_RST_REST_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/A001_02_BBP_NORMAL_audio.Table
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_BBP_NORMAL_landmarksFiltered3D.csv with als patients
[1.3703853851989958 1.0228811975919503 1.0647892711254283
 1.1273454954360465 1.3177016102665657 1.4286424431424252
 1.4526411567911883 1.6407573514035805 0.6962172243479114
 1.2303055787551802 0.8745881546987496 0.3718631638601949
 0.3824262154687317 0.3836912539518735 0.27651828043112725
 1.3304827943762565 1.4587318090179924 1.6094442578414292
 1.2700482862801534 1.0949401331022142 1.045787000097293
 0.805

[0.9708921012395858 0.9028574140908157 0.8980777105714642
 1.6582918566616396 0.7565270990582005 1.8913985220615563
 2.0190909805590143 1.6777858966860966 0.8053238481845739
 0.6861035530348858 0.3728417534045071 0.7907976227912162
 0.4013706846704497 0.2848053081266224 0.09833547983550689
 0.31094066607315385 0.9441082572891862 1.2562286778425997
 1.652104397800617 0.91900872836786 1.0678664609473956 1.0617923224818162
 1.0617923224818162 1.0617923224818162 1.0965626525945997
 1.4105996157717402 1.7509768004392818 1.7140377356930445
 1.7950493091553696 1.0661392161840626 2.524811145832995
 1.9796716216124124 2.2348704745349557 1.3375943094153375
 1.234854775421399 1.3175314431415968 1.2982221021579567
 1.4106399969089922 1.1833123099630272 1.2869789546480088 2.09646293272053
 1.578182212023925 1.1272229638189795 1.4464753644670076
 1.4526814549972407 1.4526814549972407 0.9883039376165061
 0.7854656227698423 0.44577178721114596 0.4937936111369767
 0.6080521271875055 0.19385480386388562

[1.2307142981540053 1.3008438796151018 1.1537281343474262
 1.0545960339294298 0.7524302427937827 0.9228102978383491
 0.8815836090974499 0.8815836090974499 1.389879099120661
 1.3124102072881276 0.8353298159399085 0.31162594205142236
 0.2698730197577581 0.2729376575958089 0.33537701089001876
 0.04447649612184376 0.4098354882558642 0.13480956024948382
 1.2549949058997576 2.005843368090514 2.0107695589116013
 1.8921360341111744 1.3521762456189552 0.42693675422605615
 0.7221421828223072 0.6758635911980357 0.7878879679206322
 0.6817569629309714 0.6743848818905555 0.7227083233611347
 0.689999623038145 0.8280285507106899 0.42743329228094223
 0.15060667065536074 0.6086197881106852 0.6232985164662439
 0.6232985164662439 0.8875949812301064 1.073188265845514
 0.8643760208108338 0.7890916550419518 1.0212815929270445 1.17320609313592
 1.1343915736975576 0.8944942482969729 0.631724258183169
 0.6265676306033928 1.0861809926787618 1.0955244884877864
 0.8306226592504689 0.8306226592504689 0.874350872298

[0.7780566775174673 0.5863764199378436 0.7377029892606614
 1.2518271485568844 1.1526241055222648 1.1200750866136777
 1.127556728359411 1.9479452636867445 2.970883053372741 1.2383654548616845
 0.41046453695867946 1.320785765898268 1.7651986187813693
 2.1280234796014987 2.0053313160087223 1.6310704901292068
 0.07856465282384083 0.6412523597276439 0.14685309375666983
 0.17027649968313052 0.5582498360664159 0.8909699039525205
 0.9947130060391812 1.1656879456466542 1.0185541378517693
 1.0802650237010392 1.055206788696016 1.055206788696016 0.937583763517483
 1.1819017220370291 1.1789139286433228 1.1358009894491339
 1.2216745983134583 1.0509938167357824 1.141555547121122
 1.3096978308454565 0.9096767444259354 1.8244252445594746
 1.1387643550396422 0.7772579430016207 0.7618995688982594
 0.6547342960707354 0.6374718826697406 0.3270018138271683
 0.011947565096101568 0.3387998626648823 0.3830103083670073
 0.3830103083670073 0.38495138633816184 0.034959123879584234
 0.7421782364824633 0.7656052729

In [10]:
break 

SyntaxError: 'break' outside loop (<ipython-input-10-a17de20832ec>, line 1)

In [ ]:
### THE END!

In [ ]:
#finetuned
input_csv = r"E:\FaceData_\results\Stroke_vs_Controls_new_finetuned.csv" #csv file with landmarks and .TABLE info
output_folder = r"E:\FaceData_\results\Stroke_vs_Controls_new_finetuned" #where to store the results
save_results(input_csv, output_folder, metric_type)


In [ ]:
metric_type = 'stroke' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'

#pretrained
input_csv = r"E:\FaceData_\results\Stroke_vs_Controls_database_pretrained.csv" #csv file with landmarks and .TABLE info
output_folder = r"E:\FaceData_\results\Stroke_vs_Controls_database_pretrained" #where to store the results
save_results(input_csv, output_folder, metric_type)


#finetuned
input_csv = r"E:\FaceData_\results\Stroke_vs_Controls_database_finetuned.csv" #csv file with landmarks and .TABLE info
output_folder = r"E:\FaceData_\results\Stroke_vs_Controls_database_finetuned" #where to store the results
save_results(input_csv, output_folder, metric_type)